# **Tarea de Salida BIO266E - Jupyter**

In [ ]:
#@title Importación de modulos necesarios {run: "auto", display-mode: "form"}
#@markdown Al ejecutar esta celda se importarán todos los módulos necesarios para la ejecución del código

# Modulos estandar
import os
import matplotlib.pyplot as plt
import time
import glob
import plotly
import pandas as pd
import numpy as np

# Modulos para el procesamiento de imagenes
import skimage.io as io
import skimage.exposure as exposure
import skimage.measure as measure
import skimage.color as color
import skimage.morphology as morph
import skimage.filters as filters
import skimage.segmentation as segmentation
import skimage.feature as feature
import scipy.signal as sig
import scipy.stats as stats
import scipy.ndimage as ndi
import scipy.optimize as opt
from skimage import (
    data, restoration, util
)

import holoviews as hv
import bokeh.io as bk
import bokeh.models as models
from holoviews import opts

mplparams = {
    "xtick.labelsize": "small",
    "ytick.labelsize": "small",
    "figure.autolayout": False,
    "figure.figsize": (7.2, 4.45),
    "axes.titlesize" : "small",
    "axes.labelsize": "small",
    "lines.linewidth" : 1.0,
    "lines.markersize" : 2.0,
    "legend.fontsize": "small",
    "xtick.major.size" : 6,
    "ytick.major.size" : 6,
    "xtick.minor.size" : 3,
    "ytick.minor.size" : 3,
    "xtick.major.width" : 1,
    "ytick.major.width" : 1,
    "xtick.minor.width" : 1,
    "ytick.minor.width" : 1,
    "lines.markeredgewidth" : 1,
    "font.family": "serif",
    "mathtext.fontset": "dejavuserif"
}

# For correct visualization of plots the command
# hv.extension('bokeh') must go in each cell where
# we call a bokeh object
%env HV_DOC_HTML=true;
hv.extension("bokeh")
bk.output_notebook()

# Set the previously defined parameters for all plots
plt.rcParams.update(mplparams)

## Obtenemos las imagenes a analizar

In [ ]:
!wget https://raw.githubusercontent.com/AlejoArav/BIO266E/refs/heads/master/imgs/original_photo_calibration_curve.jpg
!wget https://raw.githubusercontent.com/AlejoArav/BIO266E/refs/heads/master/imgs/original_photo_saturation_curve.jpeg

## Tarea de salida

Para esta tarea deberán analizar **dos** imagenes que se utilizaron para observar la cinética de una enzima ($\beta$-galactosidasa). La primera imagen corresponde a varios tubos que se usarán para construir una curva de calibrado para la enzima. La segunda imagen corresponden a tubos donde se dejó correr una reacción enzimática de la $\beta$-galactosidasa por 13 minutos con una concentración de sustrato distinta.

Usted deberá:

- Cargar las imagenes utilizando los nombres de archivos (1 pt).
- Escoger un transecto en el eje Y para analizar la intensidad de cada tubo (1 pt).
- Obtener los parámetros cinéticos a partir de la segunda imagen ($V_{max}$ y $K_m$) (1 pts).
- Presentar el gráfico de la curva de calibrado y de la curva de Michaelis Menten, incluyendo una breve discusión acerca de si estos parámetros se asemejan o no a los rangos determinados experimentalmente (2 pts).

**Tenga en consideración que la mayoría del código ya está escrito, sin embargo es labor de usted ver dónde cambiar ciertas variables para que se ejecute correctamente y obtener resultados esperados.**

In [ ]:
# Para cargar las imagenes debe reemplazar los signos de interrogación por el nombre de sus imagenes (recuerde incluir la extensión)
imagen_calibracion = io.imread("/content/original_photo_calibration_curve.jpg")
imagen_saturacion = io.imread("/content/original_photo_saturation_curve.jpeg")

In [ ]:
#@title Ejecute esta celda para ver sus imagenes

hv.extension("bokeh")
# Utilizamos la función de holoviews RGB para visualizar imágenes
imagen_cal = hv.RGB(imagen_calibracion, bounds=[0, 0, imagen_calibracion.shape[1], imagen_calibracion.shape[0]]).opts(
    title="Imagen curva de calibrado", xlabel="Pixeles en X", ylabel="Pixeles en Y",
)
rgbVals_cal = hv.Image(imagen_cal, ["x", "y"], ["r", "g", "b"], bounds=[0, 0, imagen_calibracion.shape[1], imagen_calibracion.shape[0]]).opts(
    alpha=0, tools=["hover"]
)
imagen_final_cal = (imagen_cal * rgbVals_cal).opts(width=imagen_calibracion.shape[1], height=imagen_calibracion.shape[0])

# Ahora para la imagen de saturación
imagen_sat = hv.RGB(imagen_saturacion, bounds=[0, 0, imagen_saturacion.shape[1], imagen_saturacion.shape[0]]).opts(
    title="Imagen curva de saturacion", xlabel="Pixeles en X", ylabel="Pixeles en Y",
)
rgbVals_sat = hv.Image(imagen_sat, ["x", "y"], ["r", "g", "b"], bounds=[0, 0, imagen_saturacion.shape[1], imagen_saturacion.shape[0]]).opts(
    alpha=0, tools=["hover"]
)
imagen_final_sat = (imagen_sat * rgbVals_sat).opts(width=imagen_saturacion.shape[1], height=imagen_saturacion.shape[0])

ambasImagenes = (imagen_final_cal + imagen_final_sat).opts(toolbar="right")

# Llamamos al objeto para visualizarlo
ambasImagenes.cols(1)

In [ ]:
#@title Definición de funciones necesarias para el procesamiento de las imágenes {run: "auto", display-mode: "form"}
#@markdown Al ejecutar esta celda se definirán funciones auxiliares para el procesamiento de las imágenes.
#@markdown Estas funciones le ayudarán a encontrar los peaks de intensidad de los tubos en las imágenes.

def processingPipelineCalibration(image):

    """
    Image processing for betagalactosidase assay using colorimetry
    """

    #invert the image (for use with white backgrounds)
    image = util.invert(image)
    #select blue channel (use blue channel if colorimetry is from semi-transparent
    #to yellow)
    image = image[:,:,2]
    #remove background
    background = restoration.rolling_ball(image, radius=35)
    final_img = image - background
    #apply gaussian
    final_img = filters.gaussian(final_img, sigma=2)

    return final_img

def processingPipelineSaturation(image):

    """
    Processing for SATURATION curve image
    """

    # Invert and select blue channel
    image = util.invert(image)[:,:,2]

    # Median and Gaussian
    final_img = filters.median(image, np.ones((10,10)))
    final_img = filters.gaussian(final_img, sigma=2)

    return final_img

def findPeaks(filtered_image, transect, distance_between_peaks, savGolIntensity=61):

    #smooth using savgol
    smoothed_transect = sig.savgol_filter(filtered_image[transect], savGolIntensity, 3)
    #find peaks
    peaks, _ = sig.find_peaks(smoothed_transect, distance=distance_between_peaks)

    return smoothed_transect, peaks

def interpolarONF(intensidad):

    """
    Función que nos retornará un valor de CONCENTRACIÓN de oNF a partir
    de un valor determinado de INTENSIDAD medida en una muestra determinada

    La variable intensidad puede ser un solo número o puede ser un arreglo de
    números, permitiendo analizar varios datos de una vez.
    """

    return (intensidad - intercepto_curvaCalibrado)/(pendiente_curvaCalibrado)

In [ ]:
# Acá se ejecutan las funciones para obtener las imágenes procesadas de la curva de calibración y de la curva de calibrado
procesada_calibracion = processingPipelineCalibration(imagen_calibracion)
procesada_saturacion = processingPipelineSaturation(imagen_saturacion)

### A continuación deberá definir la altura (en el eje Y) de los transectos y la distancia entre los peaks de intensidad (en número de pixeles) de su imagen de calibración y de saturación. A modo de prueba, pruebe una distancia entre peaks de 25 pixeles y vea los resultados

In [ ]:
# Con esta función encontraremos los peaks de intensidad de los tubos a un determinado transecto en Y
# Además se debe definir una variable que indique la distancia mínima entre los tubos (para no obtener peaks repetidos de intensidad)

CAL_altura_de_transecto = None#@param {type:"integer"}
CAL_distancia_entre_peaks = None#@param {type:"integer"}

# Una vez definida la altura del transecto y la distancia entre los peaks se ejecutará la función para encontrar los peaks
curva_calibracion, peaks_calibracion = findPeaks(procesada_calibracion,
                                                 imagen_calibracion.shape[0] - CAL_altura_de_transecto,
                                                 CAL_distancia_entre_peaks,
                                                 savGolIntensity=71)

# Ahora veamos los peaks en la curva de saturación

SAT_altura_de_transecto = None#@param {type:"integer"}
SAT_distancia_entre_peaks = None#@param {type:"integer"}

curva_saturacion, peaks_saturacion = findPeaks(procesada_saturacion,
                                                imagen_saturacion.shape[0] - SAT_altura_de_transecto,
                                                SAT_distancia_entre_peaks,
                                                savGolIntensity=71)

In [ ]:
#@title Ejecute esta celda para ver las imagenes procesadas y sus peaks de intensidad
#@markdown Recuerde verificar si los peaks corresponden a los tubos, y si se observa un aumento/saturación de la intensidad

# Veamos los peaks de la curva de calibrado y de saturación
fig, ax = plt.subplots(2, 2, figsize=(10, 5), dpi=300)
ax[0, 0].imshow(procesada_calibracion, cmap="inferno")
ax[0, 0].set_title("Imagen de calibrado")
ax[0, 0].plot([0, procesada_calibracion.shape[1]], [imagen_calibracion.shape[0] - CAL_altura_de_transecto,
                                                 imagen_calibracion.shape[0] - CAL_altura_de_transecto], "--", c="firebrick", lw=1.0)
ax[0, 0].axis("off")
ax[1, 0].plot(curva_calibracion, lw=1.0, c="royalblue", alpha=0.5)
ax[1, 0].plot(peaks_calibracion, curva_calibracion[peaks_calibracion], "*", c="royalblue", ms=5)
ax[1, 0].set_title("Peaks de intensidad de los tubos\n en la imagen de calibrado")
ax[1, 0].set_xlabel("Posición (pixeles)")
ax[1, 0].set_ylabel("Intensidad\n(Unidades Arbitrarias)");

ax[0, 1].imshow(procesada_saturacion, cmap="inferno")
ax[0, 1].set_title("Imagen de saturación")
ax[0, 1].plot([0, procesada_saturacion.shape[1]], [imagen_saturacion.shape[0] - SAT_altura_de_transecto,
                                                imagen_saturacion.shape[0] - SAT_altura_de_transecto], "--", c="firebrick", lw=1.0)
ax[0, 1].axis("off")
ax[1, 1].plot(curva_saturacion, lw=1.0, c="royalblue", alpha=0.5)
ax[1, 1].plot(peaks_saturacion, curva_saturacion[peaks_saturacion], "*", c="royalblue", ms=5)
ax[1, 1].set_title("Peaks de intensidad de los tubos\n en la imagen de saturación")
ax[1, 1].set_xlabel("Posición (pixeles)")
ax[1, 1].set_ylabel("Intensidad\n(Unidades Arbitrarias)");

In [ ]:
# Creamos un arreglo para guardar las concentraciones de sustrato
# No cambie estos valores, ya que corresponden a las concentraciones utilizadas
# experimentalmente

# Las concentraciones estarán en uM
sustratoCalibracion = np.array(
    [0,
    10,
    30,
    50,
    100,
    150,
    200,
    250,
    300,
    450,
    600
    ]
)

# Para la curva de saturación
sustratoSaturacion = np.array(
    [0,
     50,
     60,
     80,
     130,
     250,
     500,
     1000,
     1250]
)

In [ ]:
# Para la regresión lineal utilizaremos la función linregress del módulo
# stats de scipy.
regresion_lineal = stats.linregress(x = sustratoCalibracion,
                                    y = curva_calibracion[peaks_calibracion])

# Para acceder a estos datos de manera mas fácil, los guardaremos en variables
# Extraemos los datos de la regresión
pendiente_curvaCalibrado = regresion_lineal.slope
intercepto_curvaCalibrado = regresion_lineal.intercept
rcuadrado_curvaCalibrado = (regresion_lineal.rvalue)**2
stderr_pendiente = regresion_lineal.stderr  # Error estándar de la pendiente

# Grados de libertad para el cálculo del valor crítico de la t de student
n = len(sustratoCalibracion)
DoF = n - 2  # Grados de libertad (número de puntos de datos - número de parámetros ajustados)

# Valor crítico de t para el intervalo de confianza del 95%
t_crit = stats.t.ppf(0.975, DoF)

# Intervalo de confianza del 95% para la pendiente
pendiente_conf = stderr_pendiente * t_crit

# Cálculo del error estándar para el intercepto
mean_x = np.mean(sustratoCalibracion)
stderr_intercepto = stderr_pendiente * np.sqrt(np.sum(sustratoCalibracion**2) / (n * np.sum((sustratoCalibracion - mean_x)**2)))

# Intervalo de confianza del 95% para el intercepto
intercepto_conf = stderr_intercepto * t_crit

# Mostrar los resultados
print("La pendiente estimada es: {:.4f}".format(???))
print("Intervalo de confianza al 95% para la pendiente: [{:.4f}, {:.4f}]".format(
    ??? - ???, ??? + ???))

print("El intercepto estimado es: {:.4f}".format(???))
print("Intervalo de confianza al 95% para el intercepto: [{:.4f}, {:.4f}]".format(
    ??? - ???,  ??? + ???))

print("El R cuadrado calculado es de: {:.4f}".format(???))

In [ ]:
# Cálculo de concentraciones estimadas a partir de la interpolación
concentraciones_estimadas = interpolarONF(curva_saturacion[peaks_saturacion])

# La tasa de reacción la obtendremos al dividir estas reacciones
# por el tiempo total de reacción (13 minutos)
velocidades_estimadas = concentraciones_estimadas/13

# Finalmente, si consideramos que el primer tubo no contiene
# sustrato, podemos considerarlo como nuestra velocidad = 0
# y restar ese valor de velocidad de todos nuestros valores
# ya que corresponderia a nuestro ruido de fondo
velocidades_estimadas = velocidades_estimadas - velocidades_estimadas[0]

### En la celda de abajo usted debe definir la ecuación de michaelis menten (recuerde los símbolos de operación que vimos y los parametros que tiene la ecuación)

In [ ]:
def michaelisMenten(S, Vmax, Km):
    """
    Función que calcula la velocidad de reacción en función de la concentración
    de sustrato, utilizando la ecuación de Michaelis-Menten.
    """

    MM = (Vmax * S) / (Km + S)

    return MM

In [ ]:
# Ajuste de la curva
popt, pcov = opt.curve_fit(michaelisMenten, sustratoSaturacion, velocidades_estimadas, bounds=(0, np.inf))

# Extracción de parámetros
Vmax = popt[0]
Km = popt[1]

# Cálculo de errores de los parámetros (desviación estándar)
perr = np.sqrt(np.diag(pcov))
Vmax_err = perr[0]
Km_err = perr[1]

# Cálculo del 95% de confianza de los parámetros
DoF = len(sustratoSaturacion) - 2  # Número de datos - número de parámetros ajustados
t_crit = stats.t.ppf(0.975, DoF)
Vmax_conf = Vmax_err * t_crit
Km_conf = Km_err * t_crit

# Imprimir resultados
print("PARÁMETROS CINÉTICOS ESTIMADOS POR AJUSTE NO-LINEAL A MICHAELIS-MENTEN")
print(f"Vmax: {???:.3f} uM oNF/min")
print(f"Km: {???:.3f} uM ONPG\n")

print("INTERVALO DE CONFIANZA DEL 95% PARA LOS PARÁMETROS")
print(f"Vmax: [{max(0, ??? - ???):.3f}, {??? + ???:.3f}] uM oNF/min")
print(f"Km: [{max(0, ??? - ???):.3f}, {??? + ???:.3f}] uM ONPG\n")

## Acá encontrará los códigos para los gráficos!

In [ ]:
# Crear figura y ejes
fig, ax = plt.subplots(figsize=(5, 3), dpi=300)

# Datos experimentales
ax.plot(
    sustratoCalibracion,
    curva_calibracion[peaks_calibracion],
    "o",
    c="slateblue",
    label="Datos experimentales",
    alpha=0.35
)

# Predicción de la regresión lineal
y_pred = pendiente_curvaCalibrado * sustratoCalibracion + intercepto_curvaCalibrado
ax.plot(
    sustratoCalibracion,
    y_pred,
    "-.",
    c="slateblue",
    label="y = {:.6f}x + {:.6f}".format(pendiente_curvaCalibrado, intercepto_curvaCalibrado)+"\n"+r"R$^2$ = {:.6f}".format(rcuadrado_curvaCalibrado),
    lw=1.0
)

# Parámetros del grafico - Debe definir un nombre apropiado para el gráfico y rotular los ejes correspondientes
ax.set_title(???)
ax.set_xlabel(???)
ax.set_ylabel(???)

# Agregar la leyenda
ax.legend(loc='center left',
          bbox_to_anchor=(1, 0.5),
          title='Leyenda',
          frameon=False);

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3), dpi=300)
# Para los datos experimentales
ax.plot(
    sustratoSaturacion,
    velocidades_estimadas,
    "o",
    c="slateblue",
    label="Velocidades iniciales experimentales",
    alpha=0.35
)
# Para el ajuste no lineal
ax.plot(
    np.linspace(0, np.max(sustratoSaturacion), 1000),
    michaelisMenten(np.linspace(0, np.max(sustratoSaturacion), 1000), *popt),
    "-.",
    c="slateblue",
    label="y = {:.2f}x/(x + {:.2f})".format(popt[0], popt[1]),
)

# Parámetros del gráfico - Debe definir un nombre apropiado para el gráfico y rotular los ejes correspondientes
ax.set_title(???)
ax.set_xlabel(???)
ax.set_ylabel(???)

# Agregar la leyenda
ax.legend(loc='center left',
            bbox_to_anchor=(1, 0.5),
            title='Leyenda',
            frameon=False);